In [1]:
import gzip
import pickle
from tensorflow import keras
import numpy as np
from tensorflow.keras.layers import Dense, Input, Flatten, Conv2D, BatchNormalization, \
                                    Activation, Dropout, MaxPooling2D

def evaluate(path, model):
    X, y = pickle.load(gzip.open(path, 'rb'))
    y[y != 0] -= 2
    X = X / 255.
    acc = np.mean(model(X).numpy().argmax(axis=1) == y)
    return acc


In [3]:
path = 'flatland_train.data'
X,y = pickle.load(gzip.open(path,'rb')) 

# data pre-proc
y[y != 0] -= 2
X = X.reshape (X.shape[0],50,50,1) / 255.

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3,random_state = 42)

DNN modelis

In [5]:
# model = keras.models.Sequential()
# model.add(keras.layers.Flatten(input_shape=[50, 50]))
# model.add(keras.layers.Dense(64, activation="relu"))
# model.add(keras.layers.Dense(32, activation="relu"))
# model.add(keras.layers.Dense(10, activation="softmax"))
# model.compile(loss="sparse_categorical_crossentropy",
#               optimizer="Adam",
#               metrics=["accuracy"])
# model.summary()

CNN modelis

In [11]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=[50, 50, 1]))
model.add(keras.layers.BatchNormalization(axis=-1)) 
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2),strides=2))
model.add(keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.BatchNormalization(axis=-1)) 
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2),strides=2))
model.add(keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.BatchNormalization(axis=-1)) 
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2),strides=2))
model.add(keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.BatchNormalization(axis=-1)) 
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2),strides=2))
model.add(Flatten())
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(10, activation='softmax'))
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="Adam",
              metrics=["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 48, 48, 32)        320       
                                                                 
 batch_normalization_3 (Batc  (None, 48, 48, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 24, 24, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 22, 22, 64)        18496     
                                                                 
 batch_normalization_4 (Batc  (None, 22, 22, 64)       256       
 hNormalization)                                                 
                                                      

In [12]:
loss = model.fit(X_train, y_train, epochs=20,validation_data=[X_test,y_test])


Epoch 1/20
219/219 [==============================] - 5s 18ms/step - loss: 0.6514 - accuracy: 0.7710 - val_loss: 5.0685 - val_accuracy: 0.1520
Epoch 2/20
219/219 [==============================] - 4s 16ms/step - loss: 0.1896 - accuracy: 0.9590 - val_loss: 4.1481 - val_accuracy: 0.2430
Epoch 3/20
219/219 [==============================] - 3s 15ms/step - loss: 0.1353 - accuracy: 0.9704 - val_loss: 0.4839 - val_accuracy: 0.8373
Epoch 4/20
219/219 [==============================] - 3s 16ms/step - loss: 0.1116 - accuracy: 0.9751 - val_loss: 0.2101 - val_accuracy: 0.9587
Epoch 5/20
219/219 [==============================] - 3s 15ms/step - loss: 0.0814 - accuracy: 0.9804 - val_loss: 0.2418 - val_accuracy: 0.9537
Epoch 6/20
219/219 [==============================] - 3s 16ms/step - loss: 0.0689 - accuracy: 0.9810 - val_loss: 0.2129 - val_accuracy: 0.9627
Epoch 7/20
219/219 [==============================] - 3s 15ms/step - loss: 0.0783 - accuracy: 0.9776 - val_loss: 0.2152 - val_accuracy: 0.9643

In [14]:
model.save('model_cnn9.h5')


In [9]:
model = keras.models.load_model('model_cnn8.h5')
evaluate('flatland_train.data', model)

0.9772

In [15]:
pred = model.predict(X_test).argmax(axis=1)
print('Accuracy on test set - {0:.02%}'.format((pred == y_test).mean()))

Accuracy on test set - 97.50%
